## About data set

There are 3 of datasets that we will use for the analysis:

1. order_dataset
order_id
* customer_id
* order_status
* order_purchase_timestamp
* order_approved_at
* order_delivered_carrier_date
* order_delivered_customer_date
* order_estimated_delivery_date

2. customers_dataset
* customer_id
* customer_unique_id
* customer_zip_code_prefix
* customer_city
* customer_state

3. order_payment_dataset
* order_id
* payment_sequential
* payment_type
* payment_installments
* payment_value

Customer segmentation is one of the most important marketing tools, because it can help a business to better understand its target audience. Segmenting customer helps in better understanding the customers and thus personalizing marketing and communication for each segment.

We will segment our customer using RFM analysis

* Recency (R) : Difference between days since last purchase and observed days.
* Frequency (F) : How many of transcaction each customer made?
* Monetary (M) : How much each customer spend ?

In [23]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from datetime import timedelta
from sklearn.cluster import KMeans

In [24]:
# load the order_dataset
df_order = pd.read_csv('orders_dataset.csv')
pd.set_option('display.max.columns',None)
# get the overview of the dataset
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [25]:

#checking info from the dataset
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [26]:
#checking number of columns and row
df_order.shape

(99441, 8)

In [27]:

# load the order_payments_dataset
df_payment = pd.read_csv('order_payments_dataset.csv')
pd.set_option('display.max.columns',None)

#get the overview of the dataset
df_payment.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [28]:
# checking info from dataset
df_payment.info()
     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [29]:
#check the number of columns and rows
df_payment.shape

(103886, 5)

In [30]:
# load the customers_dataset
df_cust = pd.read_csv('customers_dataset.csv')
pd.set_option('display.max.columns',None)

# get overview of the dataset
df_cust.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [31]:
#checking info from the dataset
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [32]:
df_cust.shape

(99441, 5)

## Cleaning the Dataset

1. Removing Irrelevant Values
From orders_dataset, we will use only this column for the analysis. The columns are :

* order_id : the unique id of order
* customer_id : unique id of customer
* order_status : status of order (eg. Delivered, Shipped, etc)
* order_purchase_timestamp : time when the orders were made
* order_delivered_customer_date : time when the orders were succesfully arrived in customer
* order_estimated_delivery_date : estimation time when the orders were succesfully delivered to customers
We will drop order_approved_at and order_delivered_carrier_date, since we only want to consider the delivered time in this analysis.

In [33]:
#drop irrelevant columns
cols_del = df_order.iloc[0:,4:6]
df_order.drop(cols_del, axis=1, inplace=True )
df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_delivered_customer_date  96476 non-null  object
 5   order_estimated_delivery_date  99441 non-null  object
dtypes: object(6)
memory usage: 4.6+ MB


In [34]:
# get the overview of the dataset
df_order.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26 00:00:00


From customers_dataset, we will use only this columns:

* customer_id : key to the orders dataset. Each order has a unique customer_id.
* customer_unique_id : unique identifier of the customer.
* customer_city : city of customer
* customer_state : the abbreviation of state location of customer



We will drop customer_zip_code_prefix.

In [35]:
#drop irrelevant columns
df_cust = df_cust.drop(columns=['customer_zip_code_prefix'])

In [36]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   customer_id         99441 non-null  object
 1   customer_unique_id  99441 non-null  object
 2   customer_city       99441 non-null  object
 3   customer_state      99441 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


From order_payment dataset, we will use all the columns:

* order_id: unique id of orders
* payment_sequential : sequence of customer payment per order_id
* payment_type : type of payment that customers used
* payment_value : number of payment


This is the data that we will use.

In [37]:
#drop irrelevant columns
df_payment = df_payment.drop(columns=['payment_installments'])

## Handling Missing Values

In [38]:
# Customer dataset

# There is no missing value in customer_dataset

#checking missing values of cust_dataset
df_cust.isna().sum()

customer_id           0
customer_unique_id    0
customer_city         0
customer_state        0
dtype: int64

In [39]:
# Payment dataset

# There is no missing value in payment_dataset

#checking missing values of payment_dataset
df_payment.isna().sum()

order_id              0
payment_sequential    0
payment_type          0
payment_value         0
dtype: int64

In [40]:
# Order dataset

# In order_dataset there are 2965 rows or 2.98% of missing value in order_delivered_customer_date columns.

# If the order_status = 'delivered', we will drop the missing value in that column.

# But if the status is not 'delivered' yet, we will fill the missing value with NaT value. Because the missing value 
# in that columns is an expected missing value. For example, if the status is still in category 'invoiced', then it 
# is certain that there is no data that can be recorded for order_delivered_customer_date since the customer have not 
# received their orders yet.

#checking missing values of order_dataset
total_missing = df_order.isna().sum()
percent_missing = df_order.isna().sum()/ len(df_order)*100
check_missing = pd.DataFrame({'column':df_order.columns, 'total_missing':total_missing, 'percent_missing':percent_missing})
check_missing

,column,total_missing,percent_missing
order_id,order_id,0,0.000000
customer_id,customer_id,0,0.000000
order_status,order_status,0,0.000000
order_purchase_timestamp,order_purchase_timestamp,0,0.000000
order_delivered_customer_date,order_delivered_customer_date,2965,2.981668
order_estimated_delivery_date,order_estimated_delivery_date,0,0.000000


In [41]:
# There is 8 rows of missing values in order_delivered_customer_date where the status is delivered. 
# Since it just only 8 rows, we will drop that rows.

# filtering rows where order_status is delivered and order_delivered_customer_date is missing
is_delivered = df_order.loc[(df_order['order_status']=='delivered') & (df_order['order_delivered_customer_date'].isna())]
is_delivered

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
3002,2d1e2d5bf4dc7227b3bfebb81328c15f,ec05a6d8558c6455f0cbbd8a420ad34f,delivered,2017-11-28 17:44:07,NaN,2017-12-18 00:00:00
20618,f5dd62b788049ad9fc0526e3ad11a097,5e89028e024b381dc84a13a3570decb4,delivered,2018-06-20 06:58:43,NaN,2018-07-16 00:00:00
43834,2ebdfc4f15f23b91474edf87475f108e,29f0540231702fda0cfdee0a310f11aa,delivered,2018-07-01 17:05:11,NaN,2018-07-30 00:00:00
79263,e69f75a717d64fc5ecdfae42b2e8e086,cfda40ca8dd0a5d486a9635b611b398a,delivered,2018-07-01 22:05:55,NaN,2018-07-30 00:00:00
82868,0d3268bad9b086af767785e3f0fc0133,4f1d63d35fb7c8999853b2699f5c7649,delivered,2018-07-01 21:14:02,NaN,2018-07-24 00:00:00
92643,2d858f451373b04fb5c984a1cc2defaf,e08caf668d499a6d643dafd7c5cc498a,delivered,2017-05-25 23:22:43,NaN,2017-06-23 00:00:00
97647,ab7c89dc1bf4a1ead9d6ec1ec8968a84,dd1b84a7286eb4524d52af4256c0ba24,delivered,2018-06-08 12:09:39,NaN,2018-06-26 00:00:00
98038,20edc82cf5400ce95e1afacc25798b31,28c37425f1127d887d7337f284080a0f,delivered,2018-06-27 16:09:12,NaN,2018-07-19 00:00:00


In [42]:
# get the index
index = is_delivered.index
index

Index([3002, 20618, 43834, 79263, 82868, 92643, 97647, 98038], dtype='int64')

In [43]:
# drop the rows based on index
df_order.drop(index,inplace=True)

In [44]:
#there are still 2957 rows of missing value, where the order_status != delivered
df_order.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    2957
order_estimated_delivery_date       0
dtype: int64

In [45]:
#fill the missing value with NaT (Not a Time)
df_order['order_delivered_customer_date']= df_order['order_delivered_customer_date'].fillna(pd.NaT)

In [46]:
# since we don't drop the missing value, still there are 2957 rows of NaT values
df_order.isna().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_delivered_customer_date    2957
order_estimated_delivery_date       0
dtype: int64

In [47]:
# the missing value now is converted to NaT
df_order.loc[df_order['order_delivered_customer_date'].isna()]

,order_id,customer_id,order_status,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,NaN,2017-05-09 00:00:00
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,NaN,2018-06-28 00:00:00
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,NaN,2018-08-21 00:00:00
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,NaN,2017-10-03 00:00:00
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,NaN,2018-02-07 00:00:00
...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaN,2018-10-01 00:00:00
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,NaN,2018-02-06 00:00:00
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaN,2018-09-27 00:00:00
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,NaN,2017-09-15 00:00:00


In [48]:
df_order.shape

(99433, 6)

## Converting Data Types

In [49]:
# Customer dataset

# The customer_id,customer_unique_id, customer_city, customer_state are changed to object.

#change customer_id,customer_unique_id, customer_city, customer_state to object.
df_cust['customer_id'] = df_cust['customer_id'].astype(str)
df_cust['customer_unique_id'] = df_cust['customer_unique_id'].astype(str)
df_cust['customer_city'] = df_cust['customer_city'].astype(str)
df_cust['customer_state'] = df_cust['customer_state'].astype(str)

In [50]:
#change the item of customer_city to capitalized in each word
df_cust['customer_city']= df_cust['customer_city'].str.title()
df_cust.head()

,customer_id,customer_unique_id,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,Franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,Sao Bernardo Do Campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,Sao Paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,Mogi Das Cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,Campinas,SP


In [52]:
# Payment dataset

# We will change payment_sequential from int to object, because they are categorical and not will be used in aggregation.

#change payment_sequential to object
df_payment['payment_sequential']= df_payment['payment_sequential'].astype(str)

df_payment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 4 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   order_id            103886 non-null  object 
 1   payment_sequential  103886 non-null  object 
 2   payment_type        103886 non-null  object 
 3   payment_value       103886 non-null  float64
dtypes: float64(1), object(3)
memory usage: 3.2+ MB
